In [1]:
import pandas as pd
import openpyxl 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
stations_list = pd.read_excel(r'G:\fresh_start\paper\code_paper\main_data\raw_data\station_list.xlsx')

In [3]:
print(stations_list)

    Station Geo_region        Station_Name  Elevation        Lat       Long
0      1016   Mountain          Sarmathang       2574  27.944561  85.595136
1      1024       Hill           Dhulikhel       1543  27.616117  85.565503
2      1036       Hill           Panchkhal        857  27.645134  85.620881
3      1103   Mountain                Jiri       1877  27.630447  86.232114
4      1123       Hill            Manthali        497  27.394703  86.061233
5      1124   Mountain               Kabre       1755  27.633333  86.133333
6      1206       Hill         Okhaldhunga       1731  27.308121  86.504225
7      1212      Terai           Phattepur        101  26.730538  86.934812
8      1219   Mountain             Salleri       2383  27.505118  86.586215
9      1222       Hill              Diktel       1612  27.212522  86.791886
10     1303   Mountain     Chainpur (East)       1277  27.292097  87.316966
11     1304       Hill           Pakhribas       1720  27.046316  87.292473
12     1307 

In [4]:
# Load the first CSV containing Max T data
max_temp_df = pd.read_csv(r'G:\fresh_start\paper\code_paper\main_data\raw_data\max_temp_base.csv')

# Load the second CSV containing Min T data
min_temp_df = pd.read_csv(r'G:\fresh_start\paper\code_paper\main_data\raw_data\min_temp_base.csv')

# Merge the two DataFrames based on Date and Station
combined_max_min_df = pd.merge(max_temp_df, min_temp_df, on=['Date', 'Station'], how='outer')

# Save the combined DataFrame to a new CSV file
combined_max_min_df.to_csv(r'G:\fresh_start\paper\code_paper\main_data\raw_data\combined_max_min_df.csv', index=False)

In [5]:
# Merge the two DataFrames on the 'Station' column
df_merged = pd.merge(combined_max_min_df, stations_list[['Station', 'Elevation', 'Lat', 'Long']], on='Station', how='left')

# Display the merged DataFrame (optional)
print(df_merged.head())

         Date  Station  Max T  Min T  Elevation        Lat       Long
0  1962-01-01     1206   14.4    3.6     1731.0  27.308121  86.504225
1  1962-01-01     1405   14.8    1.3     1744.0  27.358611  87.670000
2  1962-01-02     1206   13.1    2.8     1731.0  27.308121  86.504225
3  1962-01-02     1405   12.8    2.4     1744.0  27.358611  87.670000
4  1962-01-03     1206   13.1    3.6     1731.0  27.308121  86.504225


In [14]:
df_merged.to_csv(r'G:\fresh_start\paper\code_paper\main_data\raw_data\combined_max_min_stations_info.csv', index=False)

In [22]:
print(df_merged['Date'].max())

2022-12-31


In [20]:
    # Calculate the IQR for 'Max T' column
    Q1 = df_merged['Max T'].quantile(0.25)
    Q3 = df_merged['Max T'].quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 3.5 * IQR
    upper_bound = Q3 + 3.5 * IQR

    print(lower_bound, upper_bound)

-4.850000000000005 51.95


In [12]:
max_max_T = df_merged['Max T'].max()
min_max_T = df_merged['Min T'].max()
max_min_T = df_merged['Max T'].min()
min_min_T = df_merged['Min T'].min()
print(f"max of max {max_max_T}, max of min {min_max_T}, min of max {max_min_T}, min of min {min_min_T}"  )
# Display the oldest da 

max of max 506.1, max of min 513.0, min of max 0.0, min of min -10.0


In [13]:
# Top 5 maximum values for 'Max T'
top_5_max_max_T = df_merged['Max T'].nlargest(5)

# Top 5 minimum values for 'Max T'
top_5_min_max_T = df_merged['Max T'].nsmallest(5)

# Top 5 maximum values for 'Min T'
top_5_max_min_T = df_merged['Min T'].nlargest(5)

# Top 5 minimum values for 'Min T'
top_5_min_min_T = df_merged['Min T'].nsmallest(5)

# Print the results
print("Top 5 maximum values for 'Max T':")
print(top_5_max_max_T)

print("\nTop 5 minimum values for 'Max T':")
print(top_5_min_max_T)

print("\nTop 5 maximum values for 'Min T':")
print(top_5_max_min_T)

print("\nTop 5 minimum values for 'Min T':")
print(top_5_min_min_T)


Top 5 maximum values for 'Max T':
28589     506.1
39403     500.0
268847     45.5
205919     45.0
205937     45.0
Name: Max T, dtype: float64

Top 5 minimum values for 'Max T':
32612     0.0
242497    0.0
242548    0.0
242582    0.0
248417    1.0
Name: Max T, dtype: float64

Top 5 maximum values for 'Min T':
39403     513.0
237065     33.5
54130      30.0
201272     29.5
54050      29.1
Name: Min T, dtype: float64

Top 5 minimum values for 'Min T':
197289   -10.0
197361   -10.0
197451   -10.0
197734    -9.6
178563    -9.5
Name: Min T, dtype: float64


In [21]:
all_raw_data = pd.read_csv(r'G:\fresh_start\paper\code_paper\main_data\raw_data\combined_max_min_stations_info.csv')

In [22]:
print(all_raw_data.max())

Date          9/9/2022
Station           1419
Max T             45.5
Min T             33.5
Elevation       2574.0
Lat          27.944561
Long         87.765595
dtype: object


In [ ]:
# Group the data by 'Station' into a dictionary
initial_max_dict = dict(tuple(all_raw_data.groupby('Station')))

# Dictionaries to store the DataFrames after outlier removal
max_outlier_check_dict = {}
min_outlier_check_dict = {}

# Loop through each station's data
for station_name, df_temp in initial_max_dict.items():
    
    # Calculate the IQR for 'Max T' and 'Min T' columns
    max_Q1 = df_temp['Max T'].quantile(0.25)
    max_Q3 = df_temp['Max T'].quantile(0.75)
    max_IQR = max_Q3 - max_Q1

    min_Q1 = df_temp['Min T'].quantile(0.25)
    min_Q3 = df_temp['Min T'].quantile(0.75)
    min_IQR = min_Q3 - min_Q1

    # Define lower and upper bounds for Max T and Min T
    max_lower_bound = max_Q1 - 1.5 * max_IQR
    max_upper_bound = max_Q3 + 1.5 * max_IQR

    min_lower_bound = min_Q1 - 1.5 * min_IQR
    min_upper_bound = min_Q3 + 1.5 * min_IQR

    # Filter out data points outside the bounds for Max T
    df_max_temp_filtered = df_temp[(df_temp['Max T'] >= max_lower_bound) & (df_temp['Max T'] <= max_upper_bound)]
    
    # Store the filtered DataFrame in the dictionary for Max T
    max_outlier_check_dict[station_name] = df_max_temp_filtered

    # Filter out data points outside the bounds for Min T
    df_min_temp_filtered = df_temp[(df_temp['Min T'] >= min_lower_bound) & (df_temp['Min T'] <= min_upper_bound)]
    
    # Store the filtered DataFrame in the dictionary for Min T
    min_outlier_check_dict[station_name] = df_min_temp_filtered

# Check the first few rows of the filtered data for each station
for station, df in max_outlier_check_dict.items():
    print(f"Station: {station}")
    print("Max")
    print(df.max())
    print("\n")

for station, df in min_outlier_check_dict.items():
    print(f"Station: {station}")
    print("Min")
    print(df.min())
    print("\n")


In [20]:
import os

import pandas as pd

# List to store each station's DataFrame with the 'Station' column added
df_list = []

# Loop through each station's filtered DataFrame
for station_name, df in max_outlier_check_dict.items():
    # Add a new column for the station
    df['Station'] = station_name
    # Append the DataFrame to the list
    df_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(df_list)

# Export the combined DataFrame to a single CSV file
combined_df.to_csv('max_outlier_checked.csv', index=False)

print("CSV file has been successfully saved!")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_12836\3232769362.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Station'] = station_name


CSV file has been successfully saved!


In [8]:
print(all_raw_data.head())

       Date  Station  Max T  Min T  Elevation        Lat       Long
0  1/1/1962     1206   14.4    3.6     1731.0  27.308121  86.504225
1  1/1/1962     1405   14.8    1.3     1744.0  27.358611  87.670000
2  1/2/1962     1206   13.1    2.8     1731.0  27.308121  86.504225
3  1/2/1962     1405   12.8    2.4     1744.0  27.358611  87.670000
4  1/3/1962     1206   13.1    3.6     1731.0  27.308121  86.504225


In [ ]:
max_outlier_check_dict = {}

for station_name, df_max_temp in initial_max_dict.items():
    


    # Calculate the IQR for 'Max T' column
    Q1 = df_max_temp['Max T'].quantile(0.25)
    Q3 = df_max_temp['Max T'].quantile(0.75)
    IQR = Q3 - Q1

    # Define lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    

    # Filter out data points outside the bounds
    df_max_temp = df_max_temp[(df_max_temp['Max T'] >= lower_bound) & (df_max_temp['Max T'] <= upper_bound)]
    max_outlier_check_dict[station_name]=df_max_temp

In [7]:
max_data = all_raw_data['Min T'].max()
print(max_data)

33.5


In [ ]:
# Group the data by 'Station'
stations = all_raw_data['Station'].unique()

# Create a separate plot for each station
for station in stations:
    station_data = all_raw_data[all_raw_data['Station'] == station]
    
    plt.figure(figsize=(14, 7))
    plt.plot(station_data['Date'], station_data['Max T'], label='Max T')
    plt.plot(station_data['Date'], station_data['Min T'], label='Min T')

    # Add labels and title
    plt.xlabel('Date')
    plt.ylabel('Temperature (°C)')
    plt.title(f'Temperature Trends for Station {station}')
    plt.legend()
    plt.grid(True)
    
    # Show the plot
    plt.show()